In [ ]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_las_out_full_path, delimiter=';')

# пока nan заменяем на 0, потом надо другое что-то сделать
# las_data = las_data.fillna(0)
# Заменяем nan на ближайшее значение
las_data = las_data.ffill()


# посмотреть какие столбцы  бывают nan
#las_data.isnull().any()

In [3]:
# Выделяем таблчку с данными по геологии п получаем количество строк во фрейме
las_data_lit = las_data[las_data['lit'] > 0]

# Применить срез по данным литологии
las_data=las_data_lit

In [4]:
# переведем данные с 0 в 1 
from sklearn.preprocessing import MinMaxScaler
las_data_norm=las_data.copy()
las_data_norm['aps'] = MinMaxScaler().fit_transform(las_data['aps'])
# las_data_norm['rp'] = MinMaxScaler().fit_transform(las_data['rp'])
las_data_norm['kp'] = MinMaxScaler().fit_transform(las_data['kp'])
las_data_norm['kgl'] = MinMaxScaler().fit_transform(las_data['kgl'])
las_data_norm['kpr'] = MinMaxScaler().fit_transform(las_data['kpr'])
las_data_norm['kvo'] = MinMaxScaler().fit_transform(las_data['kvo'])
las_data_norm['kng'] = MinMaxScaler().fit_transform(las_data['kng'])
# Применить нормализацию к данным
las_data=las_data_norm

D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

В следующем блоке выделяем данные из базы для последующего обучения. Данные преобразуем к следующему виду: скважины - строки, усредненные значения по параметру с соответсвующим шагом (столбцы), количество разбиений по глубине зависит от параметра <b>split_count</b>

In [5]:
import math
# well_name_list = las_data['well_name'].value_counts().index.tolist()
well_name_list = las_data['well_name'].value_counts().index.tolist()
data_union = pd.DataFrame() 
well_i = 0

# количество разбиений, по которым получим новые данные
split_count = 10

    
for well_name in well_name_list:
    data_well_list = []
    # Выделяем даные для одной скважины
    data_well = las_data[las_data['well_name'] == well_name]    
    # высчитываем для нее характерисктики шага по глубине
    # когда берем информацию по всем данным
    start_num = 0
    end_num = data_well.shape[0]
    h_num = round((float(end_num) - float(start_num))/split_count)
    temp_start_num = start_num
    index = 1
    while (index <= split_count):

        str_index = str(index)
        if (index == split_count):
            temp_end_num = end_num
        else:
            temp_end_num = temp_start_num + h_num

        # выделяем для конкретного участка по глубине
        data_well_by_h = data_well[temp_start_num:temp_end_num]
        columns_to_show=['aps', 'rp', 'kp', 'kgl', 'kpr', 'kvo', 'kng', 'lit', 'satur', 'wc']
        # columns_to_show=['kvo', 'satur', 'wc']
        # расчитываем среднее
        data_well_by_h = data_well_by_h.pivot_table(columns_to_show, ['well_name'] ,
                                                    aggfunc='mean')
        data_well_by_h = data_well_by_h.rename(index=str, columns={"aps": 'aps' + str_index, 'rp': 'rp' + str_index, 'kp': 'kp' + str_index, 'kgl': 'kgl' + str_index, 
                                                                'kpr': 'kpr' + str_index, 'kvo': 'kvo' + str_index, 'kng': 'kng' + str_index, 'lit': 'lit'  + str_index,
                                                                'satur': 'satur' + str_index})

        data_well_list.append(data_well_by_h)
        temp_start_num = temp_end_num
        index = index + 1
        # print(temp_start_num, temp_end_num, end_num)
    
    # делаем ключи и праметры плоскими (размерности 1)
    key_list = list([])
    for elem in data_well_list:
        key_list.extend(list(elem))
    values_list = np.vstack(data_well_list)
    values_list = values_list.ravel()
    values_list = values_list.tolist()
    values_data = [values_list]
    
    # проверяем нет ли ошибок по размерам данных
    # print(len(values_data[0]), len(key_list))
    
    # Создаем плоский срез для наших данных
    temp_frame = pd.DataFrame(data = values_data,
                 columns=key_list, index = [well_i])
    temp_frame['well_name'] = well_name
    
    
    if well_i == 0:
        data_union = temp_frame
    else:
        data_union = data_union.append(temp_frame)
    
    well_i = well_i + 1
    # break
    # print('temp_frame:', temp_frame)
    # if well_i == 3:
    #     break
print("End create data_union")

End create data_union


In [8]:
data_union.head()

,aps1,kgl1,kng1,kp1,kpr1,kvo1,lit1,rp1,satur1,aps2,...,aps10,kgl10,kng10,kp10,kpr10,kvo10,lit10,rp10,satur10,well_name
0,0.544447,0.646243,0.0,0.792298,0.098923,0.856834,1.0,2.148148,2.0,0.735474,...,0.454257,0.192777,0.056612,0.340381,0.001906,0.740782,1.0,12.998143,4.200000,4R
1,0.151515,0.580275,0.0,0.757932,0.033970,0.378121,1.0,2.011180,2.0,0.151515,...,0.345662,0.266838,0.109534,0.485628,0.020067,0.419001,1.0,8.758696,2.875776,643PL
2,0.000000,0.985330,0.0,0.019886,0.000057,1.000000,1.0,8.100000,8.0,0.000000,...,0.000000,0.985330,0.000000,0.019886,0.000057,1.000000,1.0,8.100000,8.000000,335G
3,0.151515,0.784841,0.0,0.187500,0.000057,1.000000,1.0,9.000000,8.0,0.151515,...,0.151515,0.784841,0.000000,0.187500,0.000057,1.000000,1.0,9.000000,8.000000,642G
4,0.000000,0.097800,0.0,0.428977,0.000057,1.000000,1.0,5.300000,8.0,0.000000,...,0.000000,0.097800,0.000000,0.428977,0.000057,1.000000,1.0,5.300000,8.000000,337G


In [5]:
#data_for_analize[np.isnan((data_for_analize["aps1"])==True)]
# добавляем wc
water_content_data = pd.read_csv(main.data_dir + "\\res.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
del water_content_data['Дата запуска']
data_for_analize = pd.merge(data_union, water_content_data, on=main.keys_dict[main.kid_well])

# разобраться почему есть Nan
#data_for_analize = data_for_analize.fillna(0)
y = data_for_analize['wc']
well_names = data_for_analize['well_name']
data_for_analize.drop(['wc', 'well_name'], axis=1, inplace=True)

In [6]:
# округлим вектор ответов
y_r = int(10) * round(y,1)
y_r = list(map(int, y_r))

Выделим 70% выборки (x_train, y_train) под обучение и 30% будут отложенной выборкой (x_hold, y_hold). отложенная выборка никак не будет участвовать в настройке параметров моделей, на ней мы в конце, после этой настройки, оценим качество полученной модели. Параметры дерева задаем наугад

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(data_for_analize.values, y_r, test_size=0.3) #random_state=17)

D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
tree = DecisionTreeClassifier(max_depth=20, random_state=99)
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_hold)
accuracy_score(y_hold, tree_pred)

0.21153846153846154

Теперь настроим параметры дерева на кросс-валидации. Настраивать будем максимальную глубину и максимальное используемое на каждом разбиении число признаков. Суть того, как работает GridSearchCV: для каждой уникальной пары значений параметров max_depth и max_features будет проведена 5-кратная кросс-валидация и выберется лучшее сочетание параметров.

In [10]:
from sklearn.model_selection import GridSearchCV, cross_val_score
count_vals = data_for_analize.shape[1]
tree_params = {'max_depth': range(1,30), 'max_features': range(1, count_vals)}

tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(x_train, y_train)

Fitting 5 folds for each of 2581 candidates, totalling 12905 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Done 831 tasks      | elapsed:    1.9s


[Parallel(n_jobs=-1)]: Done 11631 tasks      | elapsed:   13.9s


[Parallel(n_jobs=-1)]: Done 12905 out of 12905 | elapsed:   15.3s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=99, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 30), 'max_features': range(1, 90)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [14]:
tree_grid.best_params_

{'max_depth': 6, 'max_features': 30}

In [11]:
tree_grid.best_score_

0.39669421487603307

In [12]:
accuracy_score(y_hold, tree_grid.predict(x_hold))

0.30769230769230771

In [56]:
y_calc  = tree_grid.predict(x_hold)
data_answer = pd.DataFrame()
data_answer['hold'] = y_hold
data_answer['calc'] = y_calc
data_answer['diff'] = data_answer['calc'] == data_answer['hold']
data_answer_right = data_answer[data_answer['diff'] == True]
data_answer_wrong = data_answer[data_answer['diff'] == False]
data_answer_wrong['diff'] = 1
col_to_show = ['diff']
right  = data_answer.groupby(['hold'])['diff'].agg([np.sum])
wrong  = data_answer_wrong.groupby(['hold'])['diff'].agg([np.sum])
print(wrong)
print(right)

      sum
hold     
0       1
1       2
2      12
3       6
4       7
5       5
6       1
7       2
8       1
      sum
hold     
0     0.0
1     5.0
2     5.0
3     1.0
4     2.0
5     2.0
6     0.0
7     0.0
8     0.0


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Настройка случайного леса

In [54]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=99)
print(np.mean(cross_val_score(forest, x_train, y_train, cv=5))) # 0.949

forest_params = {'max_depth': range(1,20),
'max_features': range(4,len(x_train[0]))}

forest_grid = GridSearchCV(forest, forest_params,
cv=5, n_jobs=-1, verbose=True)
 
forest_grid.fit(x_train, y_train)
# forest_grid.best_params_, forest_grid.best_score_ # ({'max_depth': 9, 'max_features': 6}, 0.951)
# accuracy_score(y_holdout, forest_grid.predict(X_holdout)) # 0.953

D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.283374958375
Fitting 5 folds for each of 1634 candidates, totalling 8170 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.4s


[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   24.9s


...

[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  6.8min


[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  8.7min


[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 10.8min


[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 13.2min


KeyboardInterrupt: 

In [44]:
forest_grid.best_params_
forest_grid.best_score_ # ({'max_depth': 9, 'max_features': 6}, 0.951)

0.32231404958677684

In [31]:
accuracy_score(y_hold, forest_grid.predict(x_hold))

0.19230769230769232